In [1]:
ENV["PYTHONPATH"]="/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";
using Crystalline, MPBUtils, SymmetryBases, JLD2, PyPlot, DelimitedFiles, PyCall, LinearAlgebra, Brillouin
shapely_point = pyimport("shapely.geometry").Point
shapely_polygon = pyimport("shapely.geometry.polygon").Polygon
cm = 1/2.54;

In [2]:
ks_sg2 = Vector{Vector{Float64}}()
N = 50
for i in 1:N+1
for j in 1:N+1
    kx = -0.5+1*(i-1)/N 
    ky = 0.5*(j-1)/N 
    push!(ks_sg2, [kx, ky])
end
end

ks_sg10 = Vector{Vector{Float64}}()
N = 50
for i in 1:N+1
for j in 1:N+1
    kx = 0.5*(i-1)/N 
    ky = 0.5*(j-1)/N 
    push!(ks_sg10, [kx, ky])
end
end

b1 = (2/sqrt(3))*[sqrt(3)/2, 1/2]
b2 = (2/sqrt(3))*[0, 1];
v1 = [0, 0] 
v2 = -1/3*b1+2/3*b2
v3 = 1/3*(b1+b2);
c = shapely_polygon([v1, v2, v3])

ks_sg16 = Vector{Vector{Float64}}()

N = 250
for i in 1:N+1
for j in 1:N+1
    kx = -1+2*(i-1)/N 
    ky = -1+2*(j-1)/N 
    c.contains(shapely_point(kx*b1+ky*b2)) || continue
    push!(ks_sg16, [kx, ky])
end
end

In [37]:
loaded_data_sg2_lightline = load("sg2_lightline.jld2")["well_separated_nodal_points"];
loaded_data_sg10_lightline = load("sg10_lightline.jld2")["well_separated_nodal_points"];
loaded_data_sg16_lightline = load("sg16_lightline.jld2")["well_separated_nodal_points"];

loaded_data_sg2 = load("sg2_nodal_data.jld2")["well_separated_nodal_points"];
loaded_data_sg10 = load("sg10_nodal_data.jld2")["well_separated_nodal_points"];
loaded_data_sg16 = load("sg16_nodal_data.jld2")["well_separated_nodal_points"];
sgs = [2, 10, 16];

In [39]:
loaded_Rs_sg = [load("../symeigs/output/sg$sg/eps1/te/sg$sg-epsid1-res64-te-input.jld2")["Rsv"] for sg in sgs];

In [40]:
function below_light_line(sample_info::Pair{Tuple{String, Integer}, Vector{Tuple{Integer, Integer, Float64}}}, sgnum::Integer, 
        restrict_mult::Vector{Int64} = [1, 2, 3], restrict_modes::Vector{<:AbstractString}=["te", "tm"])
    mode, real_id = sample_info[1]
    (mode ∈ restrict_modes) || return 0
    #Rvecs = lattice_from_mpbparams("../symeigs/input/dim2-sg$sgnum-$(real_id - 20000)-res64-$mode.sh")[1]
    Rvecs = loaded_Rs_sg[findfirst(x -> x == sgnum, sgs)][real_id - 20000]
    Gvecs = reciprocalbasis(Rvecs)
    tot = 0
    ks = if sgnum == 2 
            ks_sg2
        elseif sgnum == 10
            ks_sg10
        elseif sgnum == 16
            ks_sg16
        end
    for x in sample_info[2]
        mult_idx, k_idx, omega = x
        (mult_idx ∈ restrict_mult) || continue
        kx, ky = ks[k_idx]
        knorms = Float64[] 
        for i in -2:2
            for j in -2:2
                knorm = norm((kx + i) * Gvecs[1] + (ky + j) * Gvecs[2])
                push!(knorms, knorm)
            end
        end
        knorm = minimum(knorms)
        if omega < knorm/(2*pi)
            tot += 1
        end
    end
    return tot
end;

In [6]:
filtered_sample_infos_sg2 = filter(x -> !isempty(x[2]), loaded_data_sg2_lightline);
filtered_sample_infos_sg10 = filter(x -> !isempty(x[2]), loaded_data_sg10_lightline);
filtered_sample_infos_sg16 = filter(x -> !isempty(x[2]), loaded_data_sg16_lightline);

In [14]:
# Find total number with frequency-isolatable nodal points 
tot_count_sg2_te = count(x -> x[1][1] == "te", filtered_sample_infos_sg2)
tot_count_sg2_tm = count(x -> x[1][1] == "tm", filtered_sample_infos_sg2)

tot_count_sg10_te = count(x -> x[1][1] == "te", filtered_sample_infos_sg10)
tot_count_sg10_tm = count(x -> x[1][1] == "tm", filtered_sample_infos_sg10)

tot_count_sg16_te = count(x -> x[1][1] == "te", filtered_sample_infos_sg16)
tot_count_sg16_tm = count(x -> x[1][1] == "tm", filtered_sample_infos_sg16)

println("Total # of samples in plane group 2 with at least one frequency isolatable Dirac point within first three multiplets (TE): $(tot_count_sg2_te)")
println("Total # of samples in plane group 2 with at least one frequency isolatable Dirac point within first three multiplets (TM): $(tot_count_sg2_tm)")
println("Total # of samples in plane group 10 with at least one frequency isolatable Dirac point within first three multiplets (TE): $(tot_count_sg10_te)")
println("Total # of samples in plane group 10 with at least one frequency isolatable Dirac point within first three multiplets (TM): $(tot_count_sg10_tm)")
println("Total # of samples in plane group 16 with at least one frequency isolatable Dirac point within first three multiplets (TE): $(tot_count_sg16_te)")
println("Total # of samples in plane group 16 with at least one frequency isolatable Dirac point within first three multiplets (TM): $(tot_count_sg16_tm)")

Total # of samples in plane group 2 with at least one frequency isolatable Dirac point within first three multiplets (TE): 630
Total # of samples in plane group 2 with at least one frequency isolatable Dirac point within first three multiplets (TM): 102
Total # of samples in plane group 10 with at least one frequency isolatable Dirac point within first three multiplets (TE): 1416
Total # of samples in plane group 10 with at least one frequency isolatable Dirac point within first three multiplets (TM): 666
Total # of samples in plane group 16 with at least one frequency isolatable Dirac point within first three multiplets (TE): 931
Total # of samples in plane group 16 with at least one frequency isolatable Dirac point within first three multiplets (TM): 365


In [22]:
# Double checking the values above 
#count(x -> x[1][1] == "tm" && !iszero(x[2]), loaded_data_sg2)
#count(x -> x[1][1] == "tm" && !iszero(x[2]), loaded_data_sg10)
#count(x -> x[1][1] == "tm" && !iszero(x[2]), loaded_data_sg16)

102

In [41]:
tot_sg2_te = 0
tot_sg2_tm = 0

prev_sg2_te = 0 
prev_sg2_tm = 0 

for sample_info in filtered_sample_infos_sg2
    num_below_light_line_te = below_light_line(sample_info, 2, [1, 2, 3], ["te"])
    num_below_light_line_tm = below_light_line(sample_info, 2, [1, 2, 3], ["tm"])
    tot_sg2_te += num_below_light_line_te
    tot_sg2_tm += num_below_light_line_tm
    prev_sg2_te += ((num_below_light_line_te > 0) ? 1 : 0)
    prev_sg2_tm += ((num_below_light_line_tm > 0) ? 1 : 0)
end

println("Total # of Dirac points per 100 samples, TE:\n$(tot_sg2_te / 100)")
println("Total # of Dirac points per 100 samples, TM:\n$(tot_sg2_tm / 100)")

println("Percentage of samples in plane group 2 with isolated nodal points below the light line, TE:\n$(prev_sg2_te / 100)")
println("Percentage of samples in plane group 2 with isolated nodal points below the light line, TM:\n$(prev_sg2_tm / 100)")

Total # of Dirac points per 100 samples, TE:
5.19
Total # of Dirac points per 100 samples, TM:
0.41
Percentage of samples in plane group 2 with isolated nodal points below the light line, TE:
5.16
Percentage of samples in plane group 2 with isolated nodal points below the light line, TM:
0.41


In [57]:
# Present as "fraction of a fraction, i.e. fraction of total number of samples with a frequency-isolatable Dirac point (among first three band multiplets)

println("Fraction of fraction, TE (given as percentage): ", round(100 * prev_sg2_te/tot_count_sg2_te, digits=1))
println("Fraction of fraction, TM (given as percentage): ", round(100 * prev_sg2_tm/tot_count_sg2_tm, digits=1))

Fraction of fraction, TE (given as percentage): 81.9
Fraction of fraction, TM (given as percentage): 40.2


In [42]:
tot_sg10_te = 0
tot_sg10_tm = 0

prev_sg10_te = 0 
prev_sg10_tm = 0 

for sample_info in filtered_sample_infos_sg10
    num_below_light_line_te = below_light_line(sample_info, 10, [1, 2, 3], ["te"])
    num_below_light_line_tm = below_light_line(sample_info, 10, [1, 2, 3], ["tm"])
    tot_sg10_te += num_below_light_line_te
    tot_sg10_tm += num_below_light_line_tm
    prev_sg10_te += ((num_below_light_line_te > 0) ? 1 : 0)
    prev_sg10_tm += ((num_below_light_line_tm > 0) ? 1 : 0)
end
println("Total # of Dirac points per 100 samples, TE:\n$(tot_sg10_te / 100)")
println("Total # of Dirac points per 100 samples, TM:\n$(tot_sg10_tm / 100)")

println("Percentage of samples in plane group 2 with isolated nodal points below the light line, TE:\n$(prev_sg10_te / 100)")
println("Percentage of samples in plane group 2 with isolated nodal points below the light line, TM:\n$(prev_sg10_tm / 100)")

Total # of Dirac points per 100 samples, TE:
2.61
Total # of Dirac points per 100 samples, TM:
2.13
Percentage of samples in plane group 2 with isolated nodal points below the light line, TE:
2.61
Percentage of samples in plane group 2 with isolated nodal points below the light line, TM:
2.13


In [56]:
println("Fraction of fraction, TE (given as percentage): ", round(100 * prev_sg10_te/tot_count_sg10_te, digits=1))
println("Fraction of fraction, TM (given as percentage): ", round(100 * prev_sg10_tm/tot_count_sg10_tm, digits=1))

Fraction of fraction, TE (given as percentage): 18.4
Fraction of fraction, TM (given as percentage): 32.0


In [58]:
tot_sg16_te = 0
tot_sg16_tm = 0

prev_sg16_te = 0 
prev_sg16_tm = 0 

for sample_info in filtered_sample_infos_sg16
    num_below_light_line_te = below_light_line(sample_info, 16, [1, 2, 3], ["te"])
    num_below_light_line_tm = below_light_line(sample_info, 16, [1, 2, 3], ["tm"])
    tot_sg16_te += num_below_light_line_te
    tot_sg16_tm += num_below_light_line_tm
    prev_sg16_te += ((num_below_light_line_te > 0) ? 1 : 0)
    prev_sg16_tm += ((num_below_light_line_tm > 0) ? 1 : 0)
end
println("Total # of Dirac points per 100 samples, TE:\n$(tot_sg16_te / 100)")
println("Total # of Dirac points per 100 samples, TM:\n$(tot_sg16_tm / 100)")

println("Percentage of samples in plane group 2 with isolated nodal points below the light line, TE:\n$(prev_sg16_te / 100)")
println("Percentage of samples in plane group 2 with isolated nodal points below the light line, TM:\n$(prev_sg16_tm / 100)")

Total # of Dirac points per 100 samples, TE:
1.36
Total # of Dirac points per 100 samples, TM:
1.23
Percentage of samples in plane group 2 with isolated nodal points below the light line, TE:
1.36
Percentage of samples in plane group 2 with isolated nodal points below the light line, TM:
1.23


In [60]:
println("Below light line sample count (TE): ", prev_sg16_te, "\nFraction of fraction, TE (given as percentage): ", round(100 * prev_sg16_te/tot_count_sg16_te, digits=1))
println("Below light line sample count (TM): ", prev_sg16_tm, "\nFraction of fraction, TM (given as percentage): ", round(100 * prev_sg16_tm/tot_count_sg16_tm, digits=1))

Below light line sample count (TE): 136
Fraction of fraction, TE (given as percentage): 14.6
Below light line sample count (TM): 123
Fraction of fraction, TM (given as percentage): 33.7


In [44]:
# Sanity Checks

In [46]:
dir = "./output/lightline/"
calcnames = readdir(dir);
sgnums = [2, 10, 16]
num_digits = 2
for (sgnum, loaded_data_lightline, ks_sg) in zip(sgnums, [loaded_data_sg2_lightline, loaded_data_sg10_lightline, loaded_data_sg16_lightline], [ks_sg2, ks_sg10, ks_sg16])
    #sgnum == 2 || continue
    sg_calcnames = filter(x -> MPBUtils.parse_sgnum(x) == sgnum, calcnames)
    for calcname in sg_calcnames
        band1_disp = readdlm(dir*calcname, ',')[6]
        id = parse(Int64, split(calcname, '-')[3])
        candidates = filter(x -> x[1][2] == id + 20000, loaded_data_lightline)
        filtered_candidates = filter(x -> !isempty(x[2]), candidates)
        println("Calcname: $calcname")
        for (key, val) in filtered_candidates
            mode = key[1]
            kidx = val[1][2]
            omega = val[1][3]
            #println(key)
           # Rvecs = lattice_from_mpbparams("../symeigs/input/dim2-sg$sgnum-$id-res64-$mode.sh")[1]
            Rvecs = loaded_Rs_sg[findfirst(x -> x == sgnum, sgs)][id]

            #if sgnum == 2
            #    println(norm(Rvecs[2]))
            #end
            Gvecs = reciprocalbasis(Rvecs)
            kx, ky = ks_sg[kidx]
            knorms = Float64[] 
            for i in -2:2
                for j in -2:2
                    knorm = norm((kx + i) * Gvecs[1] + (ky + j) * Gvecs[2])
                    push!(knorms, knorm)
                end
            end
            knorm = minimum(knorms)
            println(round(knorm/(2*pi), digits=num_digits), " ", round(band1_disp, digits=num_digits), "\n")
        end
    end
    println("\n\n")
end

Calcname: dim2-sg2-1248-res64-te-dispersion.out
0.47 0.47

Calcname: dim2-sg2-1452-res64-te-dispersion.out
0.88 0.87

Calcname: dim2-sg2-27-res64-te-dispersion.out
1.04 1.04

Calcname: dim2-sg2-2746-res64-te-dispersion.out
0.76 0.76

Calcname: dim2-sg2-3910-res64-te-dispersion.out
0.58 0.58

Calcname: dim2-sg2-5250-res64-te-dispersion.out
0.69 0.69

Calcname: dim2-sg2-7093-res64-te-dispersion.out
0.32 0.32

Calcname: dim2-sg2-7140-res64-te-dispersion.out
0.6 0.6

Calcname: dim2-sg2-8244-res64-te-dispersion.out
0.74 0.74

Calcname: dim2-sg2-9034-res64-te-dispersion.out
0.46 0.46




Calcname: dim2-sg10-1653-res64-te-dispersion.out
0.4 0.4

Calcname: dim2-sg10-2122-res64-te-dispersion.out
0.45 0.45

Calcname: dim2-sg10-3291-res64-te-dispersion.out
0.51 0.51

Calcname: dim2-sg10-3848-res64-te-dispersion.out
0.37 0.37

Calcname: dim2-sg10-480-res64-te-dispersion.out
0.27 0.27

Calcname: dim2-sg10-656-res64-te-dispersion.out
0.5 0.5

Calcname: dim2-sg10-7070-res64-te-dispersion.out
0.31 0.3